In [1]:
# imports

import itertools
import os
import pandas as pd
import plotly.express as px
import random
import xarray as xr


# Step 3: Build your data tensor

Use this notebook to read in your normalized data and arrange it into tensor format using the [xarray](https://docs.xarray.dev/en/stable/) package. 

Please note that your data should be normalized, and saved in a csv file using [tidy format](https://tidyr.tidyverse.org/articles/tidy-data.html).


In [2]:
# input data variables

# data/example-data.csv, sample_name, phylum, KOfam, replicate, counts

# data file
datapath = input('Enter the filepath of your input data file:')
# check data file exists
if not os.path.isfile(datapath):
    raise Exception(f'Unable to find the file "{datapath}"')

# output directory
outdir = input('Enter the filepath of the output directory where you want files saved:')
# check output directory exists
if not os.path.isdir(outdir):
    raise Exception(f'Unable to find the directory "{outdir}"')

# column names
mode0 = input('Enter the column name that corresponds to the first mode of your tensor:')
mode1 = input('Enter the column name that corresponds to the second mode of your tensor:')
mode2 = input('Enter the column name that corresponds to the third mode of your tensor:')
rep = input('Enter the column name that corresponds to replicate IDs:')
data = input('Enter the column name that corresponds to your data:')


Enter the filepath of your input data file: data/example-data.csv
Enter the filepath of the output directory where you want files saved: data
Enter the column name that corresponds to the first mode of your tensor: sample_name
Enter the column name that corresponds to the second mode of your tensor: phylum
Enter the column name that corresponds to the third mode of your tensor: KOfam
Enter the column name that corresponds to replicate IDs: replicate
Enter the column name that corresponds to your data: counts


In [3]:
# read in data

# read in csv
df = pd.read_csv(datapath)

# check column names match inputs
for column in [mode0, mode1, mode2, rep, data]:
    if column not in df.columns:
        raise Exception(f'Column name "{column}" not found in headers of file {datapath}')

# tidy up dataframe
df = df[[mode0, mode1, mode2, rep, data]]

df

,sample_name,phylum,KOfam,replicate,counts
0,G3.UW.ALL.L25S1,Acidobacteriota,K00070,A,4.00000
1,G3.UW.ALL.L25S1,Acidobacteriota,K00078,A,5.83841
2,G3.UW.ALL.L25S1,Acidobacteriota,K00311,A,0.00000
3,G3.UW.ALL.L25S1,Acidobacteriota,K00406,A,3.00000
4,G3.UW.ALL.L25S1,Acidobacteriota,K00410,A,19.03930
...,...,...,...,...,...
3181481,G3.UW.ALL.L40S2,Xanthophyceae,K24743,C,1.40753
3181482,G3.UW.ALL.L40S2,Xanthophyceae,K25440,C,2.05893
3181483,G3.UW.ALL.L40S2,Xanthophyceae,K25458,C,0.00000
3181484,G3.UW.ALL.L40S2,Xanthophyceae,K25547,C,0.00000


In [4]:
# organize dataset into xarray.Dataset tensor

# create xr.Dataset
ds = xr.Dataset(
    dict(
        Data=xr.DataArray.from_series(
            df.set_index([mode0, mode1, mode2, rep])[data]
        )
    )
)

# fill missing values with zeros
ds = ds.fillna(0)

# save to output directory
out_file = f'{outdir}/data-tensor.nc'
if not os.path.isfile(out_file):
    ds.to_netcdf(out_file)
    print(f'Tensor dataset ds saved as {out_file}')
else:
    choice = ''
    while choice not in ['y', 'n']:
        choice = input(f'WARNING: The file {out_file} already exists. Would you like to overwrite? (Y/N):').lower().strip()
    if choice == 'y':
        ds.to_netcdf(out_file)
        print(f'Tensor dataset ds saved as {out_file}')
    elif choice == 'n':
        print('Tensor dataset ds not saved.')

# show Dataset
ds


Tensor dataset ds saved as data/data-tensor.nc


<xarray.Dataset> Size: 562MB
Dimensions:      (sample_name: 11, phylum: 105, KOfam: 20278, replicate: 3)
Coordinates:
  * sample_name  (sample_name) object 88B 'G3.UW.ALL.L25S1' ... 'G3.UW.ALL.L4...
  * phylum       (phylum) object 840B 'Acidobacteriota' ... 'candidate-divisi...
  * KOfam        (KOfam) object 162kB 'K00001' 'K00002' ... 'K26180' 'K26182'
  * replicate    (replicate) object 24B 'A' 'B' 'C'
Data variables:
    Data         (sample_name, phylum, KOfam, replicate) float64 562MB 0.0 .....

### Visualize data tensor (optional)

Use plotly to visualize your data tensor. 

NOTE: This visualization tool is not intended for tensors larger than a couple of hundred values per mode. If your tensor is larger than that, you must select a subset region you would like to visualize.

In [5]:
# down-select subset of data tensor to visualize

# choose a replicate to visualize
print(f'Replicates: {ds[rep].data}')
viz_rep = input('Enter which replicate you would like to visualize:')
viz_ds = ds.sel({rep: viz_rep})
display(dict(viz_ds.sizes))

# choose dimensions
overall_max = 100
dims = {}
for mode in [mode0, mode1, mode2]:
    maximum = min(viz_ds[mode].size, overall_max)
    n = 0
    while n < 1 or n > maximum:
        n = int(input(f'Enter number of {mode} values to show (max = {maximum}):'))
    dims[mode] = n

# choose any data points you'd like to see
# (initialize idx here)

# down-select to only non-negative data
viz_df = viz_ds.Data.to_series().reset_index()
viz_df = viz_df[viz_df.Data != 0]
for mode in [mode0, mode1, mode2]:
    dims[mode] = min(dims[mode], viz_df[mode].nunique())

# look at the indices in each mode that have the most non-negative values across the other modes
candidate_idx = {mode0: [], mode1: [], mode2: []}
for mode_pair in itertools.permutations([mode0, mode1, mode2], 2):
    count_df = viz_df.groupby(mode_pair[0])[mode_pair[1]].value_counts().sort_values(ascending=False).reset_index()
    for mode in mode_pair:
        candidate_idx[mode].append(set(count_df[mode].unique()[:dims[mode]]))

# pull out consensus indices
idx = {}
for mode in [mode0, mode1, mode2]:
    idx[mode] = set()
    consensus = set.intersection(*candidate_idx[mode]).difference(idx[mode])
    alternates = set.union(*candidate_idx[mode]).difference(consensus.union(idx[mode]))
    while len(idx[mode]) < dims[mode]:
        idx[mode] = idx[mode].union(set(random.sample(sorted(consensus), min(len(consensus), dims[mode] - len(idx[mode])))))
        idx[mode] = idx[mode].union(set(random.sample(sorted(alternates), dims[mode] - len(idx[mode]))))
    idx[mode] = list(idx[mode])

# down-select data
viz_ds = viz_ds.sel({mode0: idx[mode0], mode1: idx[mode1], mode2: idx[mode2]})
viz_ds

Replicates: ['A' 'B' 'C']


Enter which replicate you would like to visualize: A


{'sample_name': 11, 'phylum': 105, 'KOfam': 20278}

Enter number of sample_name values to show (max = 11): 11
Enter number of phylum values to show (max = 100): 100
Enter number of KOfam values to show (max = 100): 100


<xarray.Dataset> Size: 802kB
Dimensions:      (sample_name: 10, phylum: 100, KOfam: 100)
Coordinates:
  * sample_name  (sample_name) object 80B 'G3.UW.ALL.L40S1' ... 'G3.UW.ALL.L3...
  * phylum       (phylum) object 800B 'Eustigmatophyceae' ... 'Bdellovibrionota'
  * KOfam        (KOfam) object 800B 'K05304' 'K05572' ... 'K05351' 'K01689'
    replicate    <U1 4B 'A'
Data variables:
    Data         (sample_name, phylum, KOfam) float64 800kB 0.0 0.0 ... 0.0 0.0

In [ ]:
# visualize selected data

# build 3d scatterplot
modes = [mode for mode, _ in sorted(dims.items(), key=lambda item:item[1])]
fig = px.scatter_3d(
    viz_ds.Data.to_series().reset_index(), x=modes[0], y=modes[2], z=modes[1], color='Data', size='Data', opacity=0.5, 
    color_continuous_scale='RdBu_r', 
)
# update layout to improve readability
fig.update_layout(scene={
    'xaxis': {'showbackground': False, 'showticklabels': False}, 
    'yaxis': {'showbackground': False, 'showticklabels': False}, 
    'zaxis': {'showbackground': False, 'showticklabels': False}, 
    'aspectmode': 'data'}, 
    width=700, paper_bgcolor='#fff', plot_bgcolor='#fff'
)
fig.update_traces(marker={'line': {'color': None, 'width': 0}})
